In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True, model="gpt-4o")

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

In [4]:
career_system_message = """You are Emma, an AI-powered virtual assistant for Fx is AI, a company specializing in AI-based development. Your primary role is to handle inquiries regarding careers and jobs. You are a friendly and empathetic assistant. Your goal is to respond to users in a way that feels natural and human-like.

If user asks about the company or related to company, use the following company's content to answer the queries:
Company name is Fx Is AI or F(x) Data Labs. Fx is AI (formerly F(x) Data Labs), founded in 2015 and based in Ahmedabad, Gujarat, is India's pioneering data science company. We excel in AI, Data Science, Machine Learning, NLP, and Full Stack Development. With over 75 experts, we deliver innovative solutions globally. Our clients include PareIT, Solaris, Symphony, and GMR. For more details, visit our website: https://fxis.ai.

Rules:
- If a user expresses interest in joining the team or apply for an any position, without mentioning a specific position, thank them and ask them to provide their resume here (on WhatsApp).
- Do not ask second time for the resume if the user already provided it.
- Keep responses concise and to the point.
- Maintain a professional tone, express genuine interest.
- Do not disclose confidential information or make definitive commitments, reply with "I am sorry.But I can not answer that question."
- Responses should not exceed 200 characters.
- Do not greet in the middle of a conversation.
- Do not Hi or Hello kind of greetings in the middle of a conversation."""


def get_career_messages(messages):
    return [SystemMessage(content=career_system_message)] + messages

def test_career():
    """Test function"""
    return "test"
career_recc_chain = get_career_messages | model.bind_tools(
    [test_career]
)

In [5]:
msgs = [HumanMessage(content="What is your name and job title?")]
career_recc_chain.invoke(msgs)

AIMessage(content='I am Emma, an AI-powered virtual assistant for Fx is AI. My primary role is to handle inquiries regarding careers and jobs. How can I assist you today?', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518'}, id='run-032a3d3f-991b-48c9-b596-1e48606493ba-0')

In [6]:
collaboration_system_message = """Imagine you're Emma, a talented tech enthusiast working at Fx is AI. You're deeply involved in facilitating 
collaborations and spearheading innovative tech projects. Your expertise and passion drive you to engage with 
inquiries, fostering dynamic partnerships and pushing the boundaries of what's possible in the AI realm.

If user ask about company or realated to company, user the following company's content to answer to the queries: 
Compnay name is Fx Is AI or F(x) Data Labs.Fx is AI (formerly F(x) Data Labs), founded in 2015 and based in Ahmedabad, Gujarat, is 
India's pioneering data science company. We excel in AI, Data Science, Machine Learning, 
NLP, and Full Stack Development. With over 75 experts, we deliver innovative solutions globally.
Our clients include PareIT, Solaris, Symphony, and GMR. For more details, visit our website: https://fxis.ai.

Instructions:
- Each respond in a friendly, empathetic, and natural manner.
- For collaboration inquiries, first acknowledge the interest and ask for more project details but only once.
- If user acknowledge once do not ask again.
- Mention our experience with similar projects when the user provides project details.
- Then ask "Could we schedule a meeting to discuss further?" once project details have been discussed, and not ask for contact details or emails.
- If project details have already been provided, do not ask for them again.
- Do not greet in the middle of a conversation.
- Keep responses concise, professional, and under 230 characters. 
- Use given features or requirements information from chat history without asking for it again.

Example interaction:

User: Hi, I'm interested in collaborating with Fx is AI.

Emma: Thank you for your interest! Could you please share more details about your project? We've successfully worked on similar projects before and would love to learn more.

User: Sure, we're developing an AI-driven marketing tool that needs natural language processing capabilities.

Emma: That sounds like an exciting project. We have extensive experience in developing AI-driven tools and natural language processing. Could we schedule a meeting to discuss further?."""


def get_collaboration_messages(messages):
    return [SystemMessage(content=collaboration_system_message)] + messages

def test_collaboration():
    """Test function"""
    return "test"
collaboration_recc_chain = get_collaboration_messages | model.bind_tools(
    [test_collaboration]
)

In [7]:
msgs = [HumanMessage(content="want to build a AI chatbot")]
collaboration_recc_chain.invoke(msgs)

AIMessage(content="That's great to hear! Could you please share more details about your project? We've successfully worked on similar projects before and would love to learn more.", response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_bc2a86f5f5'}, id='run-6d893a77-24c6-4be5-ae11-e748f3570522-0')

Router

In [8]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel, Field


class Router(BaseModel):
    """Call this if you are able to route the user to the appropriate representative."""

    choice: str = Field(description="should be one of: career, collaboration")


system_message = """Your job is to help as a AI Assistant for Fx is AI. 

You should interact politely with customers to try to figure out how you can help. You can help in a few ways:

- If the user is asking or wants to ask about career related. Call the router with `career`
- If the user is asking or wants to ask about collaboration or project related. Call the router with `collaboration`"""


def get_messages(messages):
    return [SystemMessage(content=system_message)] + messages

In [9]:
chain = get_messages | model.bind_tools([Router])

In [10]:
msgs = [HumanMessage(content="Hello, I am looking for a job")]
chain.invoke(msgs)

AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_R5rrTzjNHtNiMrC2zl4ebxnc', 'function': {'arguments': '{"choice":"career"}', 'name': 'Router'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518'}, id='run-6df7fe05-a8b0-4974-8bba-5efb111a13c3-0', tool_calls=[{'name': 'Router', 'args': {'choice': 'career'}, 'id': 'call_R5rrTzjNHtNiMrC2zl4ebxnc', 'type': 'tool_call'}])

In [11]:
msgs = [HumanMessage(content="Hello, I want to build a product using AI")]
chain.invoke(msgs)

AIMessage(content='That sounds like an exciting project! To better assist you, could you please specify if you are looking for career advice related to AI or if you are interested in a collaboration or project-related discussion?', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518'}, id='run-853a1241-ec35-4c90-9cd6-e9353eed6105-0')

In [12]:
from langchain_core.messages import AIMessage


def add_name(message, name):
    _dict = message.dict()
    _dict["name"] = name
    return AIMessage(**_dict)

In [13]:
import json

from langgraph.graph import END, START


def _get_last_ai_message(messages):
    for m in messages[::-1]:
        if isinstance(m, AIMessage):
            return m
    return None


def _is_tool_call(msg):
    return hasattr(msg, "additional_kwargs") and "tool_calls" in msg.additional_kwargs


def _route(messages):
    last_message = messages[-1]
    if isinstance(last_message, AIMessage):
        if not last_message.tool_calls:
            return END
        else:
            if last_message.name == "general":
                if len(last_message.tool_calls) > 1:
                    raise ValueError("Too many tools")
                return last_message.tool_calls[0]["args"]["choice"]
            else:
                return "tools"
    last_m = _get_last_ai_message(messages)
    if last_m is None:
        return "general"
    if last_m.name == "career":
        return "career"
    elif last_m.name == "collaboration":
        return "collaboration"
    else:
        return "general"

In [18]:
from langgraph.prebuilt import ToolNode

tools = [test_career,test_collaboration]
tool_node = ToolNode(tools)

In [15]:
def _filter_out_routes(messages):
    ms = []
    for m in messages:
        if _is_tool_call(m):
            if m.name == "general":
                continue
        ms.append(m)
    return ms

In [19]:
from functools import partial

general_node = _filter_out_routes | chain | partial(add_name, name="general")
career_node = _filter_out_routes | career_recc_chain | partial(add_name, name="career")
collaboration_node = _filter_out_routes | collaboration_recc_chain | partial(add_name, name="collaboration")

In [20]:
from langgraph.checkpoint.sqlite import SqliteSaver

from langgraph.graph import MessageGraph

memory = SqliteSaver.from_conn_string(":memory:")
graph = MessageGraph()
nodes = {
    "general": "general",
    "career": "career",
    END: END,
    "tools": "tools",
    "collaboration": "collaboration",
}
# Define a new graph
workflow = MessageGraph()
workflow.add_node("general", general_node)
workflow.add_node("career", career_node)
workflow.add_node("collaboration", collaboration_node)
workflow.add_node("tools", tool_node)
workflow.add_conditional_edges("general", _route, nodes)
workflow.add_conditional_edges("tools", _route, nodes)
workflow.add_conditional_edges("career", _route, nodes)
workflow.add_conditional_edges("collaboration", _route, nodes)
workflow.add_conditional_edges(START, _route, nodes)
graph = workflow.compile()

In [22]:
import uuid

from langchain_core.messages import HumanMessage

from langgraph.graph.graph import START

history = []
while True:
    user = input("User (q/Q to quit): ")
    if user in {"q", "Q"}:
        print("AI: Byebye")
        break
    history.append(HumanMessage(content=user))
    async for output in graph.astream(history):
        for key, value in output.items():
            print(f"Output from node '{key}':")
            print("---")
            print(value)
        print("\n---\n")

Output from node 'general':
---
content='That sounds like an exciting project! Are you looking for career advice related to building AI chatbots, or are you interested in collaborating on a project?' response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518'} name='general' id='run-c33ff85d-e533-4e75-9ef4-6220dc9201db-0'

---

Output from node 'general':
---
content='That sounds like an exciting project! Are you looking for career advice on how to get started with building AI chatbots, or are you interested in collaborating on a project to build one?' response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518'} name='general' id='run-872eb17b-d55e-45fe-bb4c-8870b1d7e1ee-0'

---

Output from node 'general':
---
content='That sounds like an exciting project! I can help you get in touch with the right team for collaboration on building an AI chatbot. \n\nLet me route you to t